In [11]:
# 14/05/2020, B. M. Giblin, PhD Student, Edinburgh
# Read in PSF residual quantities & calculate the terms of 'Paulin-Henriksson' model
# for \delta\xi_+ (Section 3.4 of Giblin, Heymans et al. 2020)

import numpy as np
from astropy.io import fits
import treecorr
import glob
import os
import sys
import h5py


#NorS ='S'        # "N" or "S" for North/South Fields
#print("NorS is ",NorS)

'''
####### KiDS-1000
# Read in pre-made pickled PSF catalogue (runs in ~seconds)
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/kids/kids1000_psf_catalog.h5','r') as f:
    if NorS == 'N':
        mask = f['stars/dec'][:]>-25.0
    else:
        mask = f['stars/dec'][:]<=-25.0

    RA = f['stars/ra'][:][mask]
    Dec = f['stars/dec'][:][mask]
    TPSF = f['stars/measured_T'][:][mask]
    delta_TPSF = f['stars/measured_T'][:][mask] - f['stars/model_T'][:][mask]
    e0PSF = f['stars/measured_e1'][:][mask]
    e1PSF = f['stars/measured_e2'][:][mask]
    delta_e0PSF = f['stars/measured_e1'][:][mask] - f['stars/model_e1'][:][mask]
    delta_e1PSF = f['stars/measured_e2'][:][mask] - f['stars/model_e2'][:][mask]
    
####### DES-Y3
with h5py.File('/global/cfs/cdirs/lsst/groups/WL/projects/txpipe-sys-tests/des-y3/DES_psf_y3_catalog.hdf5','r') as f:
    
    RA = f['stars/ra'][:]
    Dec = f['stars/dec'][:]
    TPSF = f['stars/measured_T'][:]
    delta_TPSF = f['stars/measured_T'][:] - f['stars/model_T'][:]
    e0PSF = f['stars/measured_e1'][:]
    e1PSF = f['stars/measured_e2'][:]
    delta_e0PSF = f['stars/measured_e1'][:] - f['stars/model_e1'][:]
    delta_e1PSF = f['stars/measured_e2'][:] - f['stars/model_e2'][:]
'''

####### HSC-Y3
# GAMA09H GAMA15H HECTOMAP VVDS WIDE12H XMM 

Field =  'XMM'
file_star = '/pscratch/sd/x/xiangchl/data/catalog/hsc_year3_shape/catalog_others/{field}_star.fits'
catalog   = fits.open(file_star.format(field=Field))[1].data


# extract ra, dec
RA, Dec   = catalog['i_ra'], catalog['i_dec']

# set flag for psf vs nonpsf stars
#idx        = np.where(catalog['psf_star']==1)[0]
#flag_used  = np.zeros(len(ra)); flag_used[idx] = 1
#idx        = np.where(catalog['fgcm_non_psf_star']==1)[0] # fgcm matched non-psf star (used)
#flag_resv  = np.zeros(len(ra)); flag_resv[idx] = 1
#idx        = np.where(catalog['non_psf_star']==1)[0]      # plain non-psf star (not used) 
#flag_resv2 = np.zeros(len(ra)); flag_resv2[idx] = 1

#if np.sum(flag_used+flag_resv2) != len(ra):
#    sys.exit("numbers of star does not add up")

# mask -- here we are not masking anything
#mask     = np.full(len(ra), True)

# extract star_e1, star_e2
star_mxx = catalog['i_sdssshape_shape11']
star_myy = catalog['i_sdssshape_shape22']
star_mxy = catalog['i_sdssshape_shape12']
e0PSF  = 0.5*(star_mxx-star_myy)/(star_mxx+star_myy)
e1PSF  = 0.5*2*star_mxy/(star_mxx+star_myy)
TPSF   = (star_mxx+star_myy)

# extract model_e1, model_e2
psf_mxx  = catalog['i_sdssshape_psf_shape11']
psf_myy  = catalog['i_sdssshape_psf_shape22']
psf_mxy  = catalog['i_sdssshape_psf_shape12']
psf_e1   = 0.5*(psf_mxx-psf_myy)/(psf_mxx+psf_myy)
psf_e2   = 0.5*2*psf_mxy/(psf_mxx+psf_myy)
psf_T    = (psf_mxx+psf_myy)

delta_TPSF = TPSF - psf_T
delta_e0PSF= e0PSF - psf_e1
delta_e1PSF = e1PSF - psf_e2

# i might want to get rid of the factor of 2 for e2 quantities here.. havent decided yet

ThBins=9
min_sep=0.5   # arcmin
max_sep=300.  # arcmin                                                                                        
bin_slop=0.05 #0.1/(np.log(max_sep/min_sep)/float(ThBins))
metric='Arc'

def Run_TreeCorr(ra,dec,y1,y2,z1,z2):
    cat1 = treecorr.Catalog(ra=ra, dec=dec,
                ra_units='degrees', dec_units='degrees',
		g1=y1, g2=y2)
    cat2 = treecorr.Catalog(ra=ra, dec=dec,
                ra_units='degrees', dec_units='degrees',
                g1=z1, g2=z2)    
    gg = treecorr.GGCorrelation(min_sep=min_sep, max_sep=max_sep,
                bin_slop=bin_slop, nbins=ThBins,
                metric=metric, sep_units='arcmin')
    gg.process(cat1,cat2)
    return gg.meanr, gg.xip, gg.xim, gg.weight




# Just calculate the PH terms across the whole data set.
print("Calculating PH terms")
print("On term 1")
meanr1, php1, phm1, weight1 = Run_TreeCorr( RA, Dec,
                                            e0PSF*delta_TPSF, e1PSF*delta_TPSF,
                                            e0PSF*delta_TPSF, e1PSF*delta_TPSF )

print("On term 2")
meanr2, php2, phm2, weight2 = Run_TreeCorr( RA, Dec, 
                                            e0PSF*delta_TPSF, e1PSF*delta_TPSF,
                                            delta_e0PSF*TPSF, delta_e1PSF*TPSF )
print("On term 3")
meanr3, php3, phm3, weight3 = Run_TreeCorr( RA, Dec,
                                            delta_e0PSF*TPSF, delta_e1PSF*TPSF,
                                            delta_e0PSF*TPSF, delta_e1PSF*TPSF )
#meanr3, php3, phm3, weight3 = Run_TreeCorr( RA, Dec,
#                                            delta_e0PSF, delta_e1PSF,
#                                            delta_e0PSF, delta_e1PSF)

print("saving file....")

np.savetxt(f'PHterms/{Field}/ph1_HSCY3.dat', np.c_[meanr1, php1, phm1, weight1],
           header='# mean-theta [arcmin], ph1_p, ph1_m, weight')
np.savetxt(f'PHterms/{Field}/ph2_HSCY3.dat', np.c_[meanr2, php2, phm2, weight2],
           header='# mean-theta [arcmin], ph2_p, ph2_m, weight')
np.savetxt(f'PHterms/{Field}/ph3_HSCY3.dat', np.c_[meanr3, php3, phm3, weight3],
           header='# mean-theta [arcmin], ph3_p, ph3_m, weight')

print("done!")
#np.savetxt('Rhoterms/rho_KiDS_%s.dat'%(NorS), np.c_[meanr3, php3, phm3, weight3],
#           header='# mean-theta [arcmin], ph3_p, ph3_m, weight')

Calculating PH terms
On term 1
On term 2
On term 3
saving file....
done!
